In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [70]:
import requests
from bs4 import BeautifulSoup

url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 30
page_size = 100

reviews = []
overall_stars = []
date = []
country = []
seat_comfort = []
cabin_service = []
food_beverages = []
ground_serv = []
recommendation = []

for i in range(1, pages + 1):

    review_url = f"{url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"
    resp = requests.get(review_url)

    cont = resp.content
    pars_cont = BeautifulSoup(cont, 'html.parser')
    for par in pars_cont.find_all("div", {"class":"text_content"}):
        reviews.append(par.get_text())

    for par in pars_cont.find_all("div", {"class":"rating-10"}):
        try:
            overall_stars.append(par.span.text)
        except:
            print(f"Missing rating star on page {i}")
            overall_stars.append("None")

    for par in pars_cont.find_all("h3"):
        date.append(par.time.text)
    
    for par in pars_cont.find_all("h3"):
        country.append(par.span.next_sibling.text.strip( " ()"))
    
    for table in pars_cont.find_all("table", {"class":"review-ratings"}):
        for row in table.find_all("tr"):
            rating_name = row.find("td", {"class":"review-rating-header"}).text.strip()
            rating_star = row.find("td", {"class":"review-rating-stars"})
            rating_value = row.find("td", {"class": "review-value"})

            if rating_star is None:
                rating_star = "None"
            else:
                rating_star = len(rating_star.find_all('span', class_='star fill'))

            if rating_value is None:
                rating_value = "None"
            else:
                rating_value = rating_value.text.strip() 

            # Append the rating name and value to respective lists
            if rating_name == "Seat Comfort":
                seat_comfort.append(rating_star)
            elif rating_name == "Cabin Staff Service": 
                cabin_service.append(rating_star)
            elif rating_name == "Food & Beverages":
                food_beverages.append(rating_star)
            elif rating_name == "Ground Service":
                ground_serv.append(rating_star)
            elif rating_name == "Recommended":
                recommendation.append(rating_value)



Missing rating star on page 30


In [60]:
len(reviews)

3000

In [61]:
len(overall_stars)

3030

In [62]:
len(seat_comfort)

2927

In [63]:
len(recommendation)

3000

In [66]:
len(cabin_service)

2887

In [76]:
len(ground_serv)

2670

In [75]:
len(food_beverages)

2679

In [77]:
seat_comfort = seat_comfort[:2670]
reviews = reviews[:2670]
ground_serv = ground_serv[:2670]
cabin_service = cabin_service[:2670]
recommendation = recommendation[:2670]
overall_stars = overall_stars[:2670]
food_beverages = food_beverages[:2670]
date = date[:2670]
country = country[:2670]

In [78]:
df = pd.DataFrame({"reviews": reviews, "rating_star": overall_stars, "date": date, "country": country, "seat_rating": seat_comfort, "staff_rating": cabin_service, "food_beverage_rating": food_beverages, "ground_service_rating": ground_serv, "recommendation": recommendation})

In [80]:
df.head()

,reviews,rating_star,date,country,seat_rating,staff_rating,food_beverage_rating,ground_service_rating,recommendation
0,✅ Trip Verified | I booked our first out of c...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,8th June 2023,United States,3,4,3,1,no
1,✅ Trip Verified | My husband and I booked seat...,1,7th June 2023,United Kingdom,4,4,5,5,no
2,"Not Verified | May 3, flew from Seattle to He...",4,6th June 2023,United States,4,4,1,2,no
3,✅ Trip Verified | I went through three flights...,1,6th June 2023,Greece,2,5,4,5,no
4,✅ Trip Verified | An excellent flight in Club ...,2,6th June 2023,United Kingdom,4,5,3,4,yes


In [82]:
df.to_csv("BA_reviews.csv")